In [ ]:
%ls

In [ ]:
import pandas as pd
import numpy as np
import json
import re
import os

from indic_transliteration import sanscript
from indic_transliteration.sanscript import transliterate


def short_pos(word):
    if 'No' in word:
        res = '_CN'
    else:
        d = re.findall('\d+',word)[0]
        res = '_C'+d
    return res

!mkdir '.\NeCTIS_XLMR Data'

def format_data(root):
    
lst = []
for path, subdirs, files in os.walk(root):
    for name in files:
        lst.append(os.path.join(path, name))
outfile_lst = [re.sub('NeCTIS Model','NeCTIS_XLMR',x) for x in lst]
outfile_lst = [re.sub('.txt','.conllu',x) for x in outfile_lst]
i = 0
# print(lst)
# print(outfile_lst)
for j in range(len(lst)):
    print(lst[j],'-->',outfile_lst[j])
for file in lst:
    with open(file) as f:
        lines = f.readlines()
    newlines = []
    new_cols = []
    x = 0
    a = 'CompNo'
    for line in lines:
        if line != '\n':
            col = line.strip().split('\t')
            x += 1
            temp = col[2]
            if col[2] == 'CompNo':
#                 col[2] = col[2]
                a,b = col[2],col[5]
            elif col[5] == 'Comp_root':
                a,b = col[2],col[5]
                col[2] = col[2]+'_End'
            else:
                if a == 'CompNo' or b == 'Comp_root':
                    a,b = col[2],col[5]
                    col[2] = col[2]+'_Start'
                else:
                    a,b = col[2],col[5]
                    col[2] = col[2]+'_Middle'
#             print(temp)
            if 'DUMMY' not in col:
                new_cols.append([col[0],transliterate(col[1],sanscript.SLP1,sanscript.DEVANAGARI)+short_pos(col[2]),'_',temp,'_','_',col[4],col[5],'_',col[2]])
            else:
                new_cols.append([col[0],col[1],'_',temp,'_','_',col[4],col[5],'_',col[2]])
        else:
#             new_cols2 = [row+[str(x)] if 'DUMMY' not in row else row+['0'] for row in new_cols]
#             new_cols2 = [row+[row[3]] for row in new_cols]
            new_cols2 = new_cols
#             print(len(new_cols2[0]))
            new_lines = ['\t'.join(x) for x in new_cols2]
            new_lines.append('\n')
            new_cols = []
            a = 'CompNo'
            x = 0
            for l in new_lines:
                newlines.append(l)
    with open(outfile_lst[i],'w') as o:
        for line in newlines:
            if line!= '\n':
                o.write(line+'\n')
            else:
                o.write(line)
    i += 1

if __name__=='__main__':
    root = "./NeCTIS Model Data"
    